## Generate the data access code

click "Insert to code" and choose "Insert SparkSession DataFrame"on "loan_sub_new.csv" 

In [10]:
//Convert label into label indices using the StringIndexer

import org.apache.spark.ml.feature.StringIndexer

val label_stringIdx = new StringIndexer().setInputCol("default").setOutputCol("label")

var df_data_1 = label_stringIdx.fit(dfData1).transform(dfData1)

df_data_1.printSchema()
df_data_1.show()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- policy_code: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- collections_12_mths_ex_med: integer (nullable = true)
 |-- acc_now_delinq: intege

In [11]:
// One-hot encoder for all strings

import org.apache.spark.ml.feature.OneHotEncoder

val categoricalColumns = Array("term", "home_ownership", "verification_status", "purpose", "addr_state")

for (categoricalCol <- categoricalColumns)
{
   var strIndex =  new StringIndexer().setInputCol(categoricalCol).setOutputCol(categoricalCol+"Index")
   var  oneHotEncoder = new OneHotEncoder().setInputCol(categoricalCol+"Index").setOutputCol(categoricalCol+"classVec")
            
   df_data_1 = strIndex.fit(df_data_1).transform(df_data_1)
   df_data_1 = oneHotEncoder.transform(df_data_1)  
}

df_data_1.printSchema()
df_data_1.show()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- policy_code: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- collections_12_mths_ex_med: integer (nullable = true)
 |-- acc_now_delinq: intege

In [12]:
import org.apache.spark.ml.feature.VectorAssembler

val numericCols = Array("loan_amnt", "int_rate", "annual_inc", "dti", "delinq_2yrs", "mths_since_last_delinq", "total_acc", "inq_last_6mths", "mths_since_last_record", "open_acc", "pub_rec", "collections_12_mths_ex_med", "acc_now_delinq")

val assemblerInputs = categoricalColumns.map(c => c + "classVec") ++ numericCols

val assembler = new VectorAssembler().
  setInputCols(assemblerInputs).
  setOutputCol("features")

df_data_1 = assembler.transform(df_data_1)

df_data_1.printSchema()
df_data_1.show()

root
 |-- id: integer (nullable = true)
 |-- member_id: integer (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- policy_code: integer (nullable = true)
 |-- application_type: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- mths_since_last_delinq: integer (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- mths_since_last_record: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- collections_12_mths_ex_med: integer (nullable = true)
 |-- acc_now_delinq: intege

In [13]:
df_data_1 = df_data_1.select("id","label", "features")
df_data_1.printSchema()
df_data_1.show()

root
 |-- id: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

+-------+-----+--------------------+
|     id|label|            features|
+-------+-----+--------------------+
|1068545|  0.0|(82,[0,3,6,22,69,...|
|1062177|  0.0|(82,[0,1,4,22,69,...|
|1066424|  0.0|(82,[0,3,4,13,30,...|
|1064908|  0.0|(82,[0,2,4,6,21,6...|
|1064623|  0.0|(82,[0,2,6,20,69,...|
|1062384|  0.0|(82,[0,1,4,12,39,...|
|1061842|  0.0|(82,[2,5,6,32,69,...|
|1060372|  0.0|(82,[0,2,4,6,43,6...|
|1052333|  0.0|(82,[0,1,5,6,35,6...|
|1046355|  0.0|(82,[0,2,6,27,69,...|
|1051570|  0.0|(82,[0,1,4,12,41,...|
|1051355|  1.0|(82,[0,1,6,20,69,...|
|1051000|  0.0|(82,[0,1,4,6,35,6...|
|1051194|  0.0|(82,[0,2,4,6,37,6...|
|1045034|  0.0|(82,[0,2,12,27,69...|
|1044439|  0.0|(82,[0,2,6,51,69,...|
|1044042|  1.0|(82,[2,5,6,26,69,...|
|1042610|  0.0|(82,[1,5,8,26,69,...|
|1037138|  1.0|(82,[2,6,25,69,70...|
|1040635|  0.0|(82,[0,2,5,6,22,6...|
+-------+-----+----------------

In [14]:
val Array(trainingData, testData) = df_data_1.randomSplit(Array(0.7, 0.3), seed = 824)
val trainCount = trainingData.count()
val testCount =  testData.count()

println(s"train:$trainCount, test:$testCount")

train:11735, test:4998


In [18]:
// LogisticRegression: you can try other classifiers
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8).setFitIntercept(true)

In [19]:
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val paramGrid = new ParamGridBuilder().addGrid(lr.regParam, Array(1.0,0.3,0.1, 0.03,0.01,0.0)).addGrid(lr.elasticNetParam, Array(0.0)).build()

val evaluator = new BinaryClassificationEvaluator()

val cv = new CrossValidator().setEstimator(lr).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(2)
  
val cvModel = cv.fit(trainingData)
evaluator.evaluate(cvModel.transform(testData))

0.6788396787835826

## Final Model

In [21]:
import org.apache.spark.ml.classification.LogisticRegressionModel
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row

val bestModel =  cvModel.bestModel.asInstanceOf[LogisticRegressionModel]

println("Model Intercept = "+ bestModel.intercept)
val coefficients = bestModel.coefficients.toArray

case class Record(weightFeature: Double)

val weightsDF = spark.createDataFrame(coefficients.map(i => Record(i)))
weightsDF.show()

Model Intercept = -1.4495233249160915
+--------------------+
|       weightFeature|
+--------------------+
|-0.11071123271753017|
|-0.03399783778646...|
|0.026553444211696486|
| 0.02223063949992781|
| 0.02876869224023332|
|0.014822597021398425|
|0.013507945371862565|
|-0.01711164608833...|
|-0.02984704550635...|
|-0.01748720484760616|
|-0.02836884641228655|
| 0.13562775882823547|
|0.052670406945680676|
|-0.05165934751904504|
|0.022204297253317296|
|0.024051635469059304|
|-0.01271836055050...|
|-0.05253508272660821|
|-0.14284567124603964|
| 0.06414050856383588|
+--------------------+
only showing top 20 rows

